In [ ]:
!pip install torch torchvision torchaudio torchsummary pycocotools opencv-python opencv-contrib-python

In [ ]:
!apt-get -qq install -y build-essential git cmake libprotobuf-dev protobuf-compiler libvulkan-dev vulkan-utils libopencv-dev

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%ls /content/gdrive/My\ Drive/

In [ ]:
!git clone https://github.com/Tencent/ncnn.git

In [ ]:
%cd ncnn

In [ ]:
!git checkout a03c1353193a172bfc22481195704731f75148d9

In [ ]:
!git submodule update --init

In [9]:
!mkdir build

In [ ]:
%cd build

In [ ]:
!cmake -DCMAKE_BUILD_TYPE=Release -DNCNN_VULKAN=ON -DNCNN_SYSTEM_GLSLANG=OFF -DNCNN_BUILD_EXAMPLES=ON ..

In [ ]:
!make

In [13]:
!cp /content/ncnn/build/tools/onnx/onnx2ncnn /usr/bin/onnx2ncnn

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/Panweitong/google-colab-tinyyolov2-model-training-script-cocopi toolkit

In [ ]:
%cd /content/toolkit/

In [ ]:
!python generate_conf.py

In [ ]:
!python train.py